## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '..')

In [5]:
import os
home_dir = os.getenv('HOME')

https://dbs.uni-leipzig.de/research/projects/object_matching/benchmark_datasets_for_entity_resolution

https://www.informatik.uni-leipzig.de/~saeedi/NCVoters_Readme.txt

```
5 party:
--------- 
5 sources. 
Each source 1000,000 entities.
There is one file per source, so totally 5 files
****************************************************
****************************************************
****************************************************
Fields:
---------- 
recId: entites with the same recId refer to the same entity.
givenname: 
surname: 
post code: 
suburb:
```

In [6]:
import glob
import csv
from tqdm.auto import tqdm

current_row_id = 0
row_dict = {}
rows_total = 5000000
cluster_attr = 'recid'

with tqdm(total=rows_total) as pbar:
    for filename in glob.glob(f'{home_dir}/Downloads/5Party-ocp20/*.csv'):
        with open(filename) as f:
            for row in csv.DictReader(f):
                row['id'] = current_row_id
                row[cluster_attr] = int(row[cluster_attr])  # convert cluster_attr to int
                row_dict[current_row_id] = row
                current_row_id += 1
                pbar.update(1)

In [7]:
row_dict[0]

{'recid': 7852009,
 'givenname': 'kadelyn',
 'surname': 'gragnani',
 'suburb': 'waxhaw',
 'postcode': '28|73',
 'id': 0}

## Preprocess

In [8]:
attr_list = ['givenname', 'surname', 'suburb', 'postcode']

In [9]:
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()[:30]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [10]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [11]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [12]:
attr_info_dict = {
    'givenname': {
        'is_multitoken': False,
        'tokenizer': None,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'surname': {
        'is_multitoken': False,
        'tokenizer': None,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'suburb': {
        'is_multitoken': False,
        'tokenizer': None,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'postcode': {
        'is_multitoken': False,
        'tokenizer': None,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [13]:
from entity_embed import build_row_encoder

row_encoder = build_row_encoder(attr_info_dict, row_dict=row_dict)
row_encoder.attr_info_dict

16:02:27 INFO:For attr='givenname', computing actual alphabet and max_str_len
16:02:30 INFO:For attr='givenname', using actual_max_str_len=16
16:02:30 INFO:For attr='surname', computing actual alphabet and max_str_len
16:02:33 INFO:actual_max_str_len=21 must be pair to enable NN pooling. Updating to 22
16:02:33 INFO:For attr='surname', using actual_max_str_len=22
16:02:33 INFO:For attr='suburb', computing actual alphabet and max_str_len
16:02:36 INFO:actual_max_str_len=21 must be pair to enable NN pooling. Updating to 22
16:02:36 INFO:For attr='suburb', using actual_max_str_len=22
16:02:36 INFO:For attr='postcode', computing actual alphabet and max_str_len
16:02:39 INFO:actual_max_str_len=9 must be pair to enable NN pooling. Updating to 10
16:02:39 INFO:For attr='postcode', using actual_max_str_len=10


{'givenname': OneHotEncodingInfo(is_multitoken=False, tokenizer=None, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=16),
 'surname': OneHotEncodingInfo(is_multitoken=False, tokenizer=None, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=22),
 'suburb': OneHotEncodingInfo(is_multitoken=False, tokenizer=None, alphabet=['0', '1', '2', '3', '4', '5', '6', '7

In [14]:
from entity_embed import DeduplicationDataModule

train_cluster_len = 1000
valid_cluster_len = 1000
test_cluster_len = 200_000
datamodule = DeduplicationDataModule(
    row_dict=row_dict,
    cluster_attr=cluster_attr,
    row_encoder=row_encoder,
    pos_pair_batch_size=45,
    neg_pair_batch_size=1225,
    row_batch_size=16,
    train_cluster_len=train_cluster_len,
    valid_cluster_len=valid_cluster_len,
    test_cluster_len=test_cluster_len,
    only_plural_clusters=True,
    log_empty_vals=False,
    random_seed=random_seed
)

## Training

In [15]:
from entity_embed import EntityEmbed

ann_k = 10
sim_threshold = 0.5
model = EntityEmbed(
    datamodule,
    ann_k=ann_k,
    sim_threshold=sim_threshold,
)

In [16]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 50
early_stop_callback = EarlyStopping(
   monitor='valid_recall',
   min_delta=0.00,
   patience=10,
   verbose=True,
   mode='max'
)
tb_log_dir = 'tb_logs'
tb_name = 'voters'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

16:02:39 INFO:GPU available: True, used: True
16:02:39 INFO:TPU available: None, using: 0 TPU cores
16:02:39 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [17]:
trainer.fit(model, datamodule)

16:02:52 WARNING:train_len + valid_len + test_len=202000 is less than len(all_cluster_id_set)=500000
16:02:54 INFO:Train pair count: 6679
16:02:54 INFO:Valid pair count: 6645
16:02:54 INFO:Test pair count: 1331364
16:02:57 INFO:
  | Name        | Type           | Params
-----------------------------------------------
0 | blocker_net | BlockerNet     | 2.5 M 
1 | losser      | NTXentLoss     | 0     
2 | miner       | BatchHardMiner | 0     
-----------------------------------------------
2.5 M     Trainable params
0         Non-trainable params
2.5 M     Total params


1

In [18]:
model.blocker_net.get_signature_weights()

{'givenname': 0.23877856135368347,
 'surname': 0.3105504512786865,
 'suburb': 0.228767991065979,
 'postcode': 0.221902996301651}

In [19]:
import gc
del trainer
gc.collect()

327

## Testing manually 

In [20]:
datamodule.setup(stage='test')

16:04:08 WARNING:train_len + valid_len + test_len=202000 is less than len(all_cluster_id_set)=500000
16:04:10 INFO:Train pair count: 6679
16:04:10 INFO:Valid pair count: 6645
16:04:10 INFO:Test pair count: 1331364


In [21]:
test_row_dict = datamodule.test_row_dict
test_vector_dict = model.predict(
    row_dict=test_row_dict,
    batch_size=16
)

In [22]:
embedding_size = model.blocker_net.embedding_size
test_true_pair_set = datamodule.test_true_pair_set

In [23]:
import gc
del model
del datamodule
gc.collect()

1305

In [24]:
assert len(test_vector_dict) == len(test_row_dict)

In [25]:
%%time

from entity_embed import ANNEntityIndex

ann_index = ANNEntityIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(test_vector_dict)
ann_index.build()

CPU times: user 14min 52s, sys: 2.66 s, total: 14min 55s
Wall time: 1min 34s


In [26]:
%%time

found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold
)

CPU times: user 22min 8s, sys: 2.11 s, total: 22min 10s
Wall time: 2min 5s


In [27]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

6.153813621883501

In [28]:
from entity_embed.evaluation import precision_and_recall

precision_and_recall(found_pair_set, test_true_pair_set)

(0.27057993003151143, 0.9995538410231912)

In [29]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

3587444

In [30]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

594

In [31]:
cos_similarity = lambda a, b: np.dot(a, b)

In [32]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_vector_dict[id_left], test_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.54677427,
 {'recid': 3259167,
  'givenname': 'lindsay',
  'surname': 'xmeltzer',
  'suburb': 'salisbury',
  'postcode': '2814q',
  'id': 4155},
 {'recid': 3259167,
  'givenname': 'lindsay',
  'surname': 'schmeltzer',
  'suburb': 'salisbury',
  'postcode': '28144',
  'id': 1742878})

(0.6822096,
 {'recid': 5584048,
  'givenname': 'angel',
  'surname': 'jones',
  'suburb': 'roanocke rapids',
  'postcode': '2787o',
  'id': 1042238},
 {'recid': 5584048,
  'givenname': 'ankel',
  'surname': 'jories',
  'suburb': 'roanoke rapids',
  'postcode': '27870',
  'id': 4042346})

(0.57757956,
 {'recid': 2991638,
  'givenname': 'anda',
  'surname': 'alzston',
  'suburb': 'durham',
  'postcode': '27707',
  'id': 48542},
 {'recid': 2991638,
  'givenname': 'wanda',
  'surname': 'alston',
  'suburb': 'durham',
  'postcode': '27707',
  'id': 1293049})

(0.57644486,
 {'recid': 947433,
  'givenname': 'mark',
  'surname': 'lerner',
  'suburb': 'cbarlotte',
  'postcode': '2821l',
  'id': 52080},
 {'recid': 947433,
  'givenname': 'marlc',
  'surname': 'leahner',
  'suburb': 'charlotte',
  'postcode': '28211',
  'id': 2051893})

(0.5978941,
 {'recid': 584874,
  'givenname': 'james',
  'surname': 'mcrae',
  'suburb': 'laurinburg',
  'postcode': '28352',
  'id': 853639},
 {'recid': 584874,
  'givenname': 'jamws',
  'surname': 'iiicrae',
  'suburb': 'laurinburg',
  'postcode': '28352',
  'id': 1016987})

(0.5607204,
 {'recid': 5601884,
  'givenname': 'maria',
  'surname': 'chaiiiba',
  'suburb': 'charlotte',
  'postcode': '28527',
  'id': 1031015},
 {'recid': 5601884,
  'givenname': 'maria',
  'surname': 'xhamba',
  'suburb': 'charlotte',
  'postcode': '282z7',
  'id': 2031014})

(0.925631,
 {'recid': 3380075,
  'givenname': 'sacqueline',
  'surname': 'hamilton',
  'suburb': 'rutherfordton',
  'postcode': '28539',
  'id': 1073481},
 {'recid': 3380075,
  'givenname': 'jacquelin',
  'surname': 'hamilton',
  'suburb': 'rutherfordton',
  'postcode': '28189',
  'id': 4073494})

(0.55688804,
 {'recid': 4709330,
  'givenname': 'briam',
  'surname': 'henderson',
  'suburb': 'norlina',
  'postcode': '27s63',
  'id': 76028},
 {'recid': 4709330,
  'givenname': 'brian',
  'surname': 'enderson',
  'suburb': 'norlins',
  'postcode': '27563',
  'id': 4076051})

(0.58952254,
 {'recid': 7510874,
  'givenname': 'axley',
  'surname': 'tompson',
  'suburb': 'stanley',
  'postcode': '28164',
  'id': 36955},
 {'recid': 7510874,
  'givenname': 'ashley',
  'surname': 'thompson',
  'suburb': 'stanley',
  'postcode': '28164',
  'id': 2642636})

(0.9143751,
 {'recid': 752819,
  'givenname': 'wanda',
  'surname': 'matthewa',
  'suburb': 'hillshorough',
  'postcode': '27278',
  'id': 20199},
 {'recid': 752819,
  'givenname': 'wanda',
  'surname': 'matthews',
  'suburb': 'hillsborough',
  'postcode': '27278',
  'id': 4278857})